# 02 - Court Detection & Homography

Module 1: Comparing classical (Hough Transform) vs deep learning (CNN keypoint) approaches.

Knowledge applied:
- Image Processing (color spaces, thresholding, morphology)
- Hough Transform
- SIFT feature matching
- Homography estimation with RANSAC

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import sys, os
sys.path.insert(0, os.path.abspath('..'))

from src.court_detection import (
    ClassicalCourtDetector,
    DeepCourtDetector,
    SIFTCourtMatcher,
    TENNIS_COURT_CORNERS,
    transform_points,
    draw_court_overlay,
)

%matplotlib inline
plt.rcParams['figure.figsize'] = (14, 8)

## 1. Load Sample Frame

In [ ]:
# Load a sample frame from video
VIDEO_PATH = '../data/raw/sample_match.mp4'

def get_sample_frame(video_path, frame_idx=100):
    cap = cv2.VideoCapture(video_path)
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)
    ret, frame = cap.read()
    cap.release()
    return frame if ret else None

# Use a test image if video not available
if os.path.exists(VIDEO_PATH):
    frame = get_sample_frame(VIDEO_PATH)
else:
    # Create a synthetic court image for testing
    frame = np.zeros((720, 1280, 3), dtype=np.uint8)
    frame[:] = (70, 130, 70)  # Green court
    # Draw white court lines
    cv2.rectangle(frame, (200, 100), (1080, 620), (255, 255, 255), 2)
    cv2.line(frame, (200, 360), (1080, 360), (255, 255, 255), 2)  # net
    cv2.rectangle(frame, (350, 100), (930, 620), (255, 255, 255), 2)  # singles
    cv2.line(frame, (350, 230), (930, 230), (255, 255, 255), 2)  # service
    cv2.line(frame, (350, 490), (930, 490), (255, 255, 255), 2)  # service
    cv2.line(frame, (640, 230), (640, 490), (255, 255, 255), 2)  # center
    print('Using synthetic court image')

plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
plt.title('Input Frame')
plt.axis('off')
plt.show()

## 2. Classical Court Detection Pipeline

Step-by-step visualization of the classical approach.

In [ ]:
detector = ClassicalCourtDetector()

# Step 1-3: Preprocessing (HSV thresholding + morphology)
mask = detector.preprocess(frame)

# Step 4: Canny edges
edges = detector.detect_edges(mask)

# Visualize preprocessing steps
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

axes[0].imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
axes[0].set_title('Original Frame')

axes[1].imshow(mask, cmap='gray')
axes[1].set_title('White Line Mask (HSV + Morphology)')

axes[2].imshow(edges, cmap='gray')
axes[2].set_title('Canny Edges')

for ax in axes:
    ax.axis('off')
plt.tight_layout()
plt.show()

In [ ]:
# Step 5-7: Hough Lines, Classification, Intersections
result = detector.detect(frame)

# Visualize detected lines
line_img = frame.copy()

# Draw horizontal lines in blue
for line in result['horizontal']:
    x1, y1, x2, y2 = line
    cv2.line(line_img, (x1, y1), (x2, y2), (255, 0, 0), 2)

# Draw vertical lines in red
for line in result['vertical']:
    x1, y1, x2, y2 = line
    cv2.line(line_img, (x1, y1), (x2, y2), (0, 0, 255), 2)

# Draw intersections
for (ix, iy) in result['intersections']:
    cv2.circle(line_img, (int(ix), int(iy)), 8, (0, 255, 0), -1)

plt.figure(figsize=(14, 8))
plt.imshow(cv2.cvtColor(line_img, cv2.COLOR_BGR2RGB))
plt.title(f"Hough Lines: {len(result['horizontal'])} horizontal (blue), "
          f"{len(result['vertical'])} vertical (red), "
          f"{len(result['intersections'])} intersections (green)")
plt.axis('off')
plt.show()

print(f"Horizontal lines: {len(result['horizontal'])}")
print(f"Vertical lines: {len(result['vertical'])}")
print(f"Intersection points: {len(result['intersections'])}")

## 3. Homography Estimation

In [ ]:
# Step 8: Compute homography
H, detection = detector.detect_and_compute_homography(frame)

if H is not None:
    print('Homography matrix H:')
    print(H)
    print()
    
    # Draw court overlay using inverse homography
    H_inv = np.linalg.inv(H)
    overlay = draw_court_overlay(frame, H_inv)
    
    plt.figure(figsize=(14, 8))
    plt.imshow(cv2.cvtColor(overlay, cv2.COLOR_BGR2RGB))
    plt.title('Court Overlay (Green = projected court model)')
    plt.axis('off')
    plt.show()
else:
    print('Could not compute homography.')
    print('Need at least 4 intersection points matching court corners.')

## 4. SIFT Feature Matching (Camera Motion)

In [ ]:
# Demonstrate SIFT matching between two frames
sift_matcher = SIFTCourtMatcher()

# Process first frame
H1 = sift_matcher.compute_frame_homography(frame)
print(f'SIFT keypoints extracted from frame')
print(f'Cumulative homography:\n{sift_matcher.cumulative_H}')

## 5. Deep Learning Court Detector

CNN-based keypoint detection (requires trained weights).

In [ ]:
# Deep learning court detector (requires trained weights)
# Uncomment after training:
#
# deep_detector = DeepCourtDetector(
#     num_keypoints=14,
#     weights_path='../models/court_detector/court_keypoint_best.pt',
#     device='cuda',
# )
# H_deep, keypoints = deep_detector.detect_and_compute_homography(frame)
#
# # Visualize predicted keypoints
# kp_img = frame.copy()
# for i, (x, y) in enumerate(keypoints):
#     cv2.circle(kp_img, (int(x), int(y)), 5, (0, 255, 0), -1)
#     cv2.putText(kp_img, str(i), (int(x)+5, int(y)-5),
#                 cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 255), 1)
#
# plt.imshow(cv2.cvtColor(kp_img, cv2.COLOR_BGR2RGB))
# plt.title('CNN Predicted Court Keypoints')
# plt.show()

print('Deep court detector ready (uncomment above after training weights)')